In [1]:
# load dataset, generate miniatures. get the overall score for the image and if it's the highest
#color it the same as the competitor.

In [2]:
import pandas as pd

In [20]:
df = pd.read_csv('dataset.csv')

In [28]:
import pandas as pd

# Assuming your dataframe is called 'df'

# Group by competitor_name and gt_source_file, and calculate mean J_value
avg_scores = df.groupby(['competitor_name', 'Gt_source_file'])['J_value'].mean().reset_index()

# Find the best competitor for each gt_source_file
best_competitors = avg_scores.loc[avg_scores.groupby('Gt_source_file')['J_value'].idxmax()]

# Sort the results for better visualization
avg_scores_pivot = avg_scores.pivot(index='Gt_source_file', 
                                  columns='competitor_name', 
                                  values='J_value')

print("\nAverage Jaccard scores for each competitor per ground truth file:")
print(avg_scores_pivot)

print("\nBest performing competitor for each ground truth file:")
print(best_competitors[['Gt_source_file', 'competitor_name', 'J_value']])

# Optional: Calculate overall average score for each competitor
overall_scores = avg_scores.groupby('competitor_name')['J_value'].mean().sort_values(ascending=False)
print("\nOverall average scores for each competitor:")
print(overall_scores)


Average Jaccard scores for each competitor per ground truth file:
competitor_name                           CALT-US   DREX-US  KIT-Sch-GE  \
Gt_source_file                                                            
inputs-2020-07/BF-C2DL-HSC/01/t0058.tif  0.952854  0.750547    0.892583   
inputs-2020-07/BF-C2DL-HSC/01/t0108.tif  0.873656  0.821078    0.831361   
inputs-2020-07/BF-C2DL-HSC/01/t0126.tif  0.856287  0.738035    0.858586   
inputs-2020-07/BF-C2DL-HSC/01/t0175.tif  0.908665  0.792961    0.826303   
inputs-2020-07/BF-C2DL-HSC/01/t0183.tif  0.919799  0.828633    0.871728   
inputs-2020-07/BF-C2DL-HSC/01/t0218.tif  0.888538  0.798949    0.846321   
inputs-2020-07/BF-C2DL-HSC/01/t0332.tif  0.902985  0.524338    0.836966   
inputs-2020-07/BF-C2DL-HSC/01/t0347.tif  0.899343  0.572179    0.869442   
inputs-2020-07/BF-C2DL-HSC/01/t0359.tif  0.888046  0.606850    0.683739   
inputs-2020-07/BF-C2DL-HSC/01/t0402.tif  0.867128  0.610859    0.860927   
inputs-2020-07/BF-C2DL-HSC/01/t04

In [29]:
len(best_competitors)

57

In [37]:
import tifffile
import cv2
import numpy as np
import os
from pathlib import Path
import pandas as pd
import math

def create_thumbnail(image_path, size=(50, 50)):
    try:
        # Read image using tifffile
        img = tifffile.imread(image_path)
        
        # Convert to 8-bit if necessary and normalize
        if img.dtype != np.uint8:
            img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
        
        # If image is grayscale, convert to RGB
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            
        # Resize image
        thumbnail = cv2.resize(img, size)
        return thumbnail
    
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

def add_colored_edge(image, color, edge_width=8):
    # Calculate the new size that maintains the same center position
    original_size = image.shape[0]  # Assuming square image
    
    # Resize the original image to make room for borders while keeping total size
    new_size = original_size - (2 * edge_width)
    if new_size <= 0:
        edge_width = original_size // 4  # Prevent too large borders
        new_size = original_size - (2 * edge_width)
    
    resized = cv2.resize(image, (new_size, new_size))
    
    # Create bordered image of the original size
    bordered = np.ones((original_size, original_size, 3), dtype=np.uint8)
    bordered = bordered * color
    
    # Calculate position to place resized image
    start_pos = edge_width
    bordered[start_pos:start_pos+new_size, start_pos:start_pos+new_size] = resized
    
    return bordered

def create_grid_image(images, grid_size=(7, 8), cell_size=50, spacing=20):
    rows, cols = grid_size
    
    # Calculate total size including spacing
    grid_height = (cell_size * rows) + (spacing * (rows - 1))
    grid_width = (cell_size * cols) + (spacing * (cols - 1))
    
    # Create blank image for the grid (white background)
    grid_img = np.ones((grid_height, grid_width, 3), dtype=np.uint8) * 255
    
    # Place images in the grid
    for idx, img in enumerate(images[:rows*cols]):
        if img is not None:
            i = idx // cols
            j = idx % cols
            
            # Calculate position with spacing
            y_start = i * (cell_size + spacing)
            x_start = j * (cell_size + spacing)
            y_end = y_start + cell_size
            x_end = x_start + cell_size
            
            # Ensure the image and the target region have the same size
            if img.shape[:2] == (cell_size, cell_size):
                grid_img[y_start:y_end, x_start:x_end] = img
    
    return grid_img

# Create output directories
thumbnail_dir = "visualizations/thumbnails"
bordered_dir = "visualizations/bordered_thumbnails"
os.makedirs(thumbnail_dir, exist_ok=True)
os.makedirs(bordered_dir, exist_ok=True)

# Assign colors to competitors (BGR format)
competitor_colors = {
    competitor: color for competitor, color in zip(
        df['competitor_name'].unique(),
        [
            (255, 128, 0),    # Blue (more vibrant)
            (0, 255, 0),      # Green (more vibrant)
            (0, 0, 255),      # Red (more vibrant)
            (0, 165, 255),    # Orange (more vibrant)
            (255, 0, 255),    # Magenta (more vibrant)
            (0, 255, 255),    # Yellow (more vibrant)
        ]
    )
}

# Get the best competitor for each source file
best_competitors = df.groupby(['Gt_source_file', 'competitor_name'])['J_value'].mean().reset_index()
best_competitors = best_competitors.loc[best_competitors.groupby('Gt_source_file')['J_value'].idxmax()]
best_competitors_dict = dict(zip(best_competitors['Gt_source_file'], best_competitors['competitor_name']))

# Lists to store thumbnails
basic_thumbnails = []
bordered_thumbnails = []

cell_size = 50  # Base size for all images
edge_width = 8  # Border width

# Process each unique source file
for source_file in df['Gt_source_file'].unique():
    # Create thumbnail
    thumbnail = create_thumbnail(source_file, size=(cell_size, cell_size))
    if thumbnail is not None:
        # Store basic thumbnail
        basic_thumbnails.append(thumbnail)
        
        # Get best competitor and corresponding color
        best_competitor = best_competitors_dict[source_file]
        color = competitor_colors[best_competitor]
        
        # Create and store bordered version
        bordered = add_colored_edge(thumbnail, color, edge_width)
        bordered_thumbnails.append(bordered)
        
        print(f"Processed {Path(source_file).name} - Best competitor: {best_competitor}")

# Create and save grid images
spacing = 20
grid_basic = create_grid_image(basic_thumbnails, cell_size=cell_size, spacing=spacing)
grid_bordered = create_grid_image(bordered_thumbnails, cell_size=cell_size, spacing=spacing)

# Save grid images
cv2.imwrite(os.path.join(thumbnail_dir, "grid_basic.png"), grid_basic)
cv2.imwrite(os.path.join(bordered_dir, "grid_bordered.png"), grid_bordered)

# Create a legend image
legend_height = 50 * len(competitor_colors)
legend_img = np.zeros((legend_height, 250, 3), dtype=np.uint8)
legend_img.fill(255)  # White background

y_position = 25
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
thickness = 1

for competitor, color in competitor_colors.items():
    cv2.putText(legend_img, competitor, (90, y_position), 
                font, font_scale, (0, 0, 0), thickness)
    # Larger color rectangle
    cv2.rectangle(legend_img, (20, y_position-20), (70, y_position+20), 
                 color, -1)
    y_position += 50

cv2.imwrite(os.path.join(bordered_dir, "legend.png"), legend_img)

print("\nProcessing complete!")
print(f"Grid images saved as:")
print(f"Basic grid: {os.path.join(thumbnail_dir, 'grid_basic.png')}")
print(f"Bordered grid: {os.path.join(bordered_dir, 'grid_bordered.png')}")
print(f"Legend saved as: {os.path.join(bordered_dir, 'legend.png')}")

Processed t0046.tif - Best competitor: CALT-US
Processed t0056.tif - Best competitor: CALT-US
Processed t0287.tif - Best competitor: CALT-US
Processed t0448.tif - Best competitor: CALT-US
Processed t0799.tif - Best competitor: CALT-US
Processed t0879.tif - Best competitor: CALT-US
Processed t1585.tif - Best competitor: CALT-US
Processed t1748.tif - Best competitor: CALT-US
Processed t0058.tif - Best competitor: CALT-US
Processed t0108.tif - Best competitor: MU-Lux-CZ
Processed t0126.tif - Best competitor: MU-Lux-CZ
Processed t0175.tif - Best competitor: MU-Lux-CZ
Processed t0183.tif - Best competitor: MU-Lux-CZ
Processed t0218.tif - Best competitor: CALT-US
Processed t0332.tif - Best competitor: MU-Lux-CZ
Processed t0347.tif - Best competitor: MU-Lux-CZ
Processed t0359.tif - Best competitor: MU-Lux-CZ
Processed t0402.tif - Best competitor: MU-Lux-CZ
Processed t0473.tif - Best competitor: MU-Lux-CZ
Processed t0475.tif - Best competitor: MU-Lux-CZ
Processed t0528.tif - Best competitor: M